In [1]:
from dotenv import load_dotenv
import os
import openai

In [2]:
load_dotenv()

True

In [3]:
API_KEY = os.getenv("api_key")

In [4]:
client = openai.OpenAI(
    api_key=API_KEY
)

In [5]:
# 그대로 사용하거나 구글 검색을 하는 API 따로 있어서 이걸 사용해도 됨
# 다양한 검색 API 존재하므로 참고

import requests
from bs4 import BeautifulSoup

def scrap_google_news(keyword: str, limit=5):
    google_search_url = 'https://www.google.com/search'

    params = {'q': keyword, 'tbm': 'nws', 'num': limit}

    headers = {
        "User-Agent":
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"
    }

    res = requests.get(google_search_url, params=params, headers=headers)

    soup = BeautifulSoup(res.content, 'html.parser')

    news_results = []
    for el in soup.select("div.SoaBEf"):
        news_results.append(
            {
                "link": el.find("a")["href"],
                "title": el.select_one("div.MBeuO").get_text(),
                "snippet": el.select_one(".GI74Re").get_text()
            }
        )

    return news_results

In [6]:
import json

results = scrap_google_news('프롬프트 엔지니어링')

print(json.dumps(results, ensure_ascii=False, indent=2))

[
  {
    "link": "https://www.apple-economy.com/news/articleView.html?idxno=72953",
    "title": "유망직종, 프롬프트 엔지니어? “쓸모없게 된다” < AI(인공지능) < ICT < 기사본문",
    "snippet": "[애플경제 이윤순 기자] 생성AI가 발전할수록 유망한 직종으로 꼽히고 있는 IT직종 중 하나가 '프롬프트 엔지니어'다. 그러나 이제 막 \n각광을 받기..."
  },
  {
    "link": "https://www.fntoday.co.kr/news/articleView.html?idxno=315945",
    "title": "디지털융합교육원 최재용 원장, '미국 밴더빌트 대학교 프롬프트엔지니어링 과정' 수료",
    "snippet": "12일 디지털융합교육원 최재용 원장은 미국 밴더빌트 대학교에서 진행된 프롬프트엔지니어링 과정을 성공적으로 마쳤다."
  },
  {
    "link": "https://www.aitimes.com/news/articleView.html?idxno=155674",
    "title": "MS \"프롬프트 엔지니어링 잘하면 미세조정 필요 없어\"",
    "snippet": "마이크로소프트(MS)가 'GPT-4'와 같은 범용 대형언어모델(LLM)로도 미세조정을 거친 전문 의학용 LLM의 성능을 뛰어넘을 수 \n있는 '프롬프트..."
  },
  {
    "link": "https://www.itworld.co.kr/news/311236",
    "title": "요즘 핫한 커리어 스킬, '프롬프트 엔지니어링' 현황과 전망",
    "snippet": "아울러 프롬프트 엔지니어는 연간 최대 33만 5,000달러라는 높은 연봉을 받는 것으로 알려졌다. 프롬프트 엔지니어링이 인간과 기계의 \n상호작용을 개선..."
  },
  {
    "link": "https://www.aitimes.kr/news/articleVi

In [7]:
def news_gpt(messages, temperature=0, max_tokens=1024):
    functions = [
        {
            "name": "scrap_google_news", # 위의 function 명
            "description": "구글에서 뉴스를 검색합니다.", # 이 설명을 통해, 이 설명에 해당하는 사용자의 요청이 있을 때, 이 함수를 호출하는 것으로 이해하고, 이를 호출해줌
            "parameters": { # 함수 호출에 필요한 파라미터 설정
                "type": "object",
                "properties": {
                    "keyword": { # 필수 파라미터
                        "type": "string",
                        "description": "구글에 검색할 키워드",
                    }
                },
                "required": ["keyword"], # required로 필수 파라미터 설정
            },
        }
    ]

    completion = client.chat.completions.create( # function calling은 chat completion만 사용 가능
        model="gpt-3.5-turbo",
        messages=messages,
        functions=functions,
        function_call="auto", # 자동으로 판별해서 결과를 넣어주도록 설정
        temperature=temperature,
        max_tokens=max_tokens,
    )

    return completion.choices[0].message

In [8]:
result = news_gpt([
    {
        "role": "user",
        "content": "세계 경제에 대한 뉴스를 요약해줘"
    }
])

dict(result)

{'content': None,
 'role': 'assistant',
 'function_call': FunctionCall(arguments='{"keyword":"세계 경제 뉴스"}', name='scrap_google_news'),
 'tool_calls': None}

In [9]:
import json

arguments = json.loads(result.function_call.arguments)

arguments

{'keyword': '세계 경제 뉴스'}

In [10]:
results = scrap_google_news(arguments['keyword'])

print(json.dumps(results, ensure_ascii=False, indent=2))

[
  {
    "link": "https://www.nocutnews.co.kr/news/6108444",
    "title": "세계 경제 질서 싹 바꿀 트럼프 2.0…심지어 숨겨 놓은 카드도?[문제는경제야,바보야]",
    "snippet": "트럼프의 美, 中 공격할 '금융' 카드 숨겨놨다…노장 라이타이저 주목▷김광석>'트럼프 이스 백' 어떻게 생각하십니까? ▷전병서> 유명한 \n구호가 하나..."
  },
  {
    "link": "https://www.newsquest.co.kr/news/articleView.html?idxno=219724",
    "title": "[윤한홍의 세계경제 진실게임] AI덕에 춤추는 기업들...오라클 주가급등에 뉴스케일파워(SMR)와 두산에너빌리티 주목",
    "snippet": "【뉴스퀘스트=윤한홍 경제에디터 】 현재 인공지능(AI)과 관련된 세계적 기업들의 주가 움직임이 매우 활발하다. 가장 최근에는 미국 \n현지시간 3월12..."
  },
  {
    "link": "https://www.mk.co.kr/news/business/10963579",
    "title": "중견련 “경제와 안보 융합을 기치로 주요 국가와 협력 강화해야”",
    "snippet": "최진식 중견련 회장, 조태열 외교장관 만나 “전방위 지원체계 고도화하는 데 힘써달라”"
  },
  {
    "link": "https://www.voakorea.com/a/7399452.html",
    "title": "[2023 연말 특집] 경제 주요 뉴스",
    "snippet": "2023년 한 해가 저물어 가고 있습니다. 생방송 여기는 워싱턴입니다, 이 시간에는 올 한해를 분야별로 재조명해 보는 연말 특집 \n준비했습니다."
  },
  {
    "link": "https://www.fnnews.com/news/202403131121520710",
    "title": "글로벌 기업·인재 몰리는 '부산진해경제자유

In [11]:
result = news_gpt([
    {
        "role": "user",
        "content": json.dumps(results, ensure_ascii=False)
    },
    {
        "role": "user",
        "content": "세계 경제에 대한 뉴스를 요약해줘. 뉴스를 종합하여 제목과 본문이 있는 새로운 블로그 글로 작성해줘."
    }
])

dict(result)

{'content': "# 세계 경제에 대한 뉴스 요약\n\n## 1. [세계 경제 질서 싹 바꿀 트럼프 2.0…심지어 숨겨 놓은 카드도?[문제는경제야,바보야]](https://www.nocutnews.co.kr/news/6108444)\n- 트럼프의 미국과 중국을 공격할 '금융' 카드를 숨겨놓고 있다는 소식이 전해졌다.\n- 노장 라이타이저가 주목받고 있으며, 트럼프의 이스 백에 대한 다양한 의견이 나오고 있다.\n\n## 2. [[윤한홍의 세계경제 진실게임] AI덕에 춤추는 기업들...오라클 주가급등에 뉴스케일파워(SMR)와 두산에너빌리티 주목](https://www.newsquest.co.kr/news/articleView.html?idxno=219724)\n- 현재 AI와 관련된 세계적 기업들의 주가 움직임이 활발하다.\n- 미국의 오라클 주가 급등과 뉴스케일파워(SMR)와 두산에너빌리티에 대한 주목이 집중되고 있다.\n\n## 3. [중견련 “경제와 안보 융합을 기치로 주요 국가와 협력 강화해야”](https://www.mk.co.kr/news/business/10963579)\n- 중견련은 경제와 안보 융합을 기치로 주요 국가와의 협력 강화를 강조했다.\n- 최진식 중견련 회장과 조태열 외교장관이 전방위 지원체계 고도화를 위해 노력하고 있다.\n\n## 4. [[2023 연말 특집] 경제 주요 뉴스](https://www.voakorea.com/a/7399452.html)\n- 2023년 한 해를 돌아보며 경제 분야별 주요 뉴스를 다시 살펴보는 연말 특집이 준비되었다.\n- 워싱턴에서 생방송으로 진행되며, 올 한 해의 경제 이슈들을 다시 한번 되짚어볼 예정이다.\n\n## 5. [글로벌 기업·인재 몰리는 '부산진해경제자유구역'](https://www.fnnews.com/news/202403131121520710)\n- 부산진해경제자유구역이 글로벌 기업과 인재들이 몰리는 곳으로 주목받고 있다.\n- 부산진해경제자유구역은 기업하기 좋고 살기 좋은 환경으

In [12]:
print(result.content)

# 세계 경제에 대한 뉴스 요약

## 1. [세계 경제 질서 싹 바꿀 트럼프 2.0…심지어 숨겨 놓은 카드도?[문제는경제야,바보야]](https://www.nocutnews.co.kr/news/6108444)
- 트럼프의 미국과 중국을 공격할 '금융' 카드를 숨겨놓고 있다는 소식이 전해졌다.
- 노장 라이타이저가 주목받고 있으며, 트럼프의 이스 백에 대한 다양한 의견이 나오고 있다.

## 2. [[윤한홍의 세계경제 진실게임] AI덕에 춤추는 기업들...오라클 주가급등에 뉴스케일파워(SMR)와 두산에너빌리티 주목](https://www.newsquest.co.kr/news/articleView.html?idxno=219724)
- 현재 AI와 관련된 세계적 기업들의 주가 움직임이 활발하다.
- 미국의 오라클 주가 급등과 뉴스케일파워(SMR)와 두산에너빌리티에 대한 주목이 집중되고 있다.

## 3. [중견련 “경제와 안보 융합을 기치로 주요 국가와 협력 강화해야”](https://www.mk.co.kr/news/business/10963579)
- 중견련은 경제와 안보 융합을 기치로 주요 국가와의 협력 강화를 강조했다.
- 최진식 중견련 회장과 조태열 외교장관이 전방위 지원체계 고도화를 위해 노력하고 있다.

## 4. [[2023 연말 특집] 경제 주요 뉴스](https://www.voakorea.com/a/7399452.html)
- 2023년 한 해를 돌아보며 경제 분야별 주요 뉴스를 다시 살펴보는 연말 특집이 준비되었다.
- 워싱턴에서 생방송으로 진행되며, 올 한 해의 경제 이슈들을 다시 한번 되짚어볼 예정이다.

## 5. [글로벌 기업·인재 몰리는 '부산진해경제자유구역'](https://www.fnnews.com/news/202403131121520710)
- 부산진해경제자유구역이 글로벌 기업과 인재들이 몰리는 곳으로 주목받고 있다.
- 부산진해경제자유구역은 기업하기 좋고 살기 좋은 환경으로 세계 물류와 비즈니스의 메카로 자리매김하고 있다.

--